In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

#데이터 불러오기
df = pd.read_csv("국민건강보험공단_건강검진정보_20221231.csv", encoding='cp949')

#트리 예시를 만들기 위해 일부 특성만 추출하기
sample_df = df[['신장(5cm단위)', '성별', '체중(5kg단위)', '음주여부']]
#샘플 데이터 중 상위 10개 추출하기
sample_df[:10]

,신장(5cm단위),성별,체중(5kg단위),음주여부
0,160,2,75,0.0
1,150,2,45,1.0
2,150,2,60,1.0
3,160,2,50,1.0
4,160,2,55,1.0
5,170,1,85,1.0
6,160,2,40,1.0
7,180,1,65,1.0
8,155,2,50,1.0
9,165,2,55,1.0


In [3]:
#데이터 전처리하기

In [5]:
#info 정보로 결측치 확인하기
print("Info 정보 확인")
sample_df.info()

Info 정보 확인
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   신장(5cm단위)  1000000 non-null  int64  
 1   성별         1000000 non-null  int64  
 2   체중(5kg단위)  1000000 non-null  int64  
 3   음주여부       999927 non-null   float64
dtypes: float64(1), int64(3)
memory usage: 30.5 MB


In [6]:
#100만 개 데이터 중 196개, 결측치가 포함된 행 삭제하기
sample = sample_df.dropna()
#결측치(null) 다시 확인하기
print("Drop 후 Info 정보 확인")
sample.info()

Drop 후 Info 정보 확인
<class 'pandas.core.frame.DataFrame'>
Int64Index: 999927 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   신장(5cm단위)  999927 non-null  int64  
 1   성별         999927 non-null  int64  
 2   체중(5kg단위)  999927 non-null  int64  
 3   음주여부       999927 non-null  float64
dtypes: float64(1), int64(3)
memory usage: 38.1 MB


In [7]:
#원학 인코딩을 위해 데이터 object 형태로 변경하기
sample = sample.astype('str')
#label(결과, Y) 생성하기
y = sample.음주여부
#음주여부 학습 데이터 구성하기
x = sample.drop('음주여부', axis=1)
#label 데이터의 편향성 확인하기
y.value_counts()

1.0    655146
0.0    344781
Name: 음주여부, dtype: int64

In [8]:
#학습/검증 데이터 분리하기
x_train, x_valid, y_train, y_valid = train_test_split(
    x,y,
    test_size = 0.2,
    shuffle=True,
    random_state=34
)

In [9]:
#의사결정나무 모델링하기

In [11]:
from sklearn.tree import DecisionTreeClassifier

#의사결정나무 모델 생성하기
dt = DecisionTreeClassifier(random_state = 1001, max_depth=2)
#의사결정나무 학습하기
dt_model = dt.fit(x_train, y_train)
#학습 데이터 정확도 확인하기
print("학습 정확도 = ", dt_model.score(x_train, y_train))
#검증 데이터 정확도 확인하기
print("검증 정확도 = ", dt_model.score(x_valid, y_valid))

학습 정확도 =  0.6985977715856544
검증 정확도 =  0.7003840268818817


In [12]:
#의사결정나무 그래프 그리기

In [14]:
import matplotlib.pyplot as plt
!pip install graphviz
import graphviz
from sklearn.tree import export_graphviz

#그래프 생성하기
tree_graph = graphviz.Source(export_graphviz(dt_model, class_names=["X","O"], feature_names=['height', 'sex', 'weight'], impurity=True, filled=True))

#그래프 출력하기
tree_graph
#그래프 저장하기
tree_graph.render('tree_depth5', format="png")

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH

In [15]:
#불순도 알아보기

In [16]:
#불순도 함수 생성하기
def gini(x):
    n = x.sum()
    gini_sum = 0
    for key in x.keys():
        gini_sum = gini_sum + (x[key] / n) * (x[key] / n)
    gini = 1 - gini_sum
    return gini
#데이터 준비하기
과일바구니1 = ['사과']*9
과일바구니2 = ['사과','바나나','사과','바나나','바나나','바나나','복숭아','복숭아','복숭아']
과일바구니3 = ['사과','바나나','사과','바나나','사과','복숭아','복숭아','사과','복숭아']

print(round(gini(pd.DataFrame(과일바구니1).value_counts()),3))
print(round(gini(pd.DataFrame(과일바구니2).value_counts()),3))
print(round(gini(pd.DataFrame(과일바구니3).value_counts()),3))

0.0
0.642
0.642
